In [1]:
# Step 1: Install Necessary Libraries
!pip install nltk scikit-learn numpy requests


Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Step 2: Import Required Libraries
import numpy as np
import nltk
import random
import string  # To process standard python strings
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
# Step 3: Download NLTK Packages
nltk.download('punkt')  # For tokenization
nltk.download('punkt_tab')  # For tokenization
nltk.download('wordnet')  # For lemmatization

# Step 4: Load the Data
with open('chatbot_data.txt', 'r', errors='ignore') as f:
    raw_text = f.read().lower()

# Step 5: Text Preprocessing
from nltk.tokenize import sent_tokenize, word_tokenize

# Step 5: Text Preprocessing
sent_tokens = sent_tokenize(raw_text)  # Convert the text to a list of sentences
word_tokens = word_tokenize(raw_text)  # Convert the text to a list of words  # Convert the text to a list of sentences
word_tokens = nltk.word_tokenize(raw_text)  # Convert the text to a list of words

lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SAHIL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\SAHIL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SAHIL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# Step 6: Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["hi", "hey", "hello", "I am glad you're talking to me!"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [13]:
# Step 7: Crypto Price Check using Gemini API
CRYPTO_KEYWORDS = ["bitcoin", "ethereum", "crypto", "price"]

def get_crypto_price(crypto):
    base_url = f"https://api.gemini.com/v1/pubticker/{crypto}usd"
    try:
        response = requests.get(base_url)
        data = response.json()
        return f"The current price of {crypto.capitalize()} is ${data['last']}"
    except:
        return "Sorry, I couldn't retrieve the price at the moment."


In [14]:
# Step 8: Generate Response

def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0:
        robo_response += "I am sorry! I don't understand you."
    else:
        robo_response += sent_tokens[idx]
    sent_tokens.remove(user_response)
    return robo_response

In [ ]:
# Step 9: Chatbot Conversation Loop
flag = True
print("Chatbot: Hello! I am an NLP chatbot with crypto price feature. Type 'bye' to exit.")

while flag:
    user_response = input("You: ").lower()
    if user_response != 'bye':
        if user_response in ['thanks', 'thank you']:
            flag = False
            print("Chatbot: You're welcome!")
        elif any(keyword in user_response for keyword in CRYPTO_KEYWORDS):
            if "bitcoin" in user_response:
                print("Chatbot: " + get_crypto_price("btc"))
            elif "ethereum" in user_response:
                print("Chatbot: " + get_crypto_price("eth"))
            else:
                print("Chatbot: Sorry, I can only check Bitcoin and Ethereum prices right now.")
        else:
            if greeting(user_response) is not None:
                print("Chatbot: " + greeting(user_response))
            else:
                print("Chatbot: " + response(user_response))
    else:
        flag = False
        print("Chatbot: Bye! Take care.")

Chatbot: Hello! I am an NLP chatbot with crypto price feature. Type 'bye' to exit.


You:  c


Chatbot: I am sorry! I don't understand you.


You:  hi


Chatbot: hi


You:  what's the price  of bitcoin


Chatbot: The current price of Btc is $80462.85


You:  and ethereum


Chatbot: The current price of Eth is $2139.92
